In [32]:
# imports
import pandas as pd
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [33]:
# load data
df = pd.read_csv("top_rated_wines.csv")
df = df[df["variety"].notna()]  # remove any NaN values as it blows up serialization
data = df.to_dict("records")

In [34]:
# create embeddings
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [35]:
# create the vector database client
vdb = QdrantClient(":memory:")  # create in-memory Qdrant instance

In [36]:
# create the collection
new_collection_name = "top_wines"
vdb.create_collection(
    collection_name=new_collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [37]:
# vectorize
vdb.upload_points(
        collection_name=new_collection_name,
        points=[
            models.PointStruct(
                id=index,
                vector=encoder.encode(content["notes"]).tolist(),
                payload=content,
            )
            for index, content in enumerate(data)
        ],
    )

In [38]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [39]:
# search locally
hits = vdb.search(
    collection_name=new_collection_name,
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3,
)
for hit in hits:
    print(hit.payload['name'], hit.payload['region'], "score:", hit.score)

Catena Zapata Argentino Vineyard Malbec 2004 Argentina score: 0.6377782384717134
Bodega Colome Altura Maxima Malbec 2012 Salta, Argentina score: 0.6179680846507815
Catena Zapata Adrianna Vineyard Malbec 2004 Argentina score: 0.6117575753165064


/var/folders/1t/stmkpc7x67x4qstrsg949vdc0000gn/T/ipykernel_47267/1440686408.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = vdb.search(


In [40]:
# naive check if embeddings are stored
vdb.scroll(
    collection_name=new_collection_name,
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="variety", match=models.MatchValue(value="Red Wine")),
        ]
    ),
    limit=3,
    with_payload=True,
    with_vectors=False,
)

([Record(id=0, payload={'name': '3 Rings Reserve Shiraz 2004', 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004. Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank maturation of the blend prior to bottling, July 2007. '}, vector=None, shard_key=None, order_value=None),
  Record(id=1, payload={'name': 'Abreu Vineyards Cappella 2007', 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Cappella is a proprietary blend of two clones of Cabernet Sauvignon with Cabernet Franc, Petit Verdot and Merlot. The gravelly soil at Cappella produces fruit that is very elegant in structure. The resulting wine exhibits beautiful purity of fru

In [42]:
# better check if embeddings are stored
checkpoints = vdb.get_collection(new_collection_name)
print(f"Collection {new_collection_name} contains {checkpoints.points_count} points.")

Collection top_wines contains 1347 points.


In [43]:
# store search results to be used by llm
search_results = [hit.payload for hit in hits]

In [47]:
# LLM interaction with openai lib
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="sk-no-key-required",
)
completion = client.chat.completions.create(
    model="llama3.1",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)},
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content=' \n\nMy top suggestion, though you might have a different taste experience or another wine suits your preferences better as everyone has their own palatable experiences. How does **Bodega Colome Altura Maxima Malbec 2012**, that earned an impressive 96 ratings look to you?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [45]:
# LLM interaction with langchain lib
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

llm = OllamaLLM(model="llama3.1")
template = """
You are chatbot, a wine specialist. 
Your top priority is to help guide users into selecting amazing wine and guide them with their requests.
Answer the question below using the conversation history and the search results content provided.
Here is the conversation history: {context}
Here are the search results: {search_results}
Question: {question}
Answer:
"""
context = ""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm
response = chain.invoke({"context": context, "search_results": str(search_results), "question": user_prompt})
print(response)

I'd be happy to help you find an amazing Malbec wine from Argentina!

Based on your interest in Malbec, I would highly recommend the **Catena Zapata Adrianna Vineyard Malbec 2004**. This wine has a rating of 97.0 and is described as "opulent, full-flavored, yet remarkably light on its feet." It's also mentioned that it will certainly evolve for a decade but is hard to resist now.

This wine is a single-vineyard Malbec from the Gualtallary district in Argentina, and it offers complex aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. I think you'll find this wine to be an exceptional representation of Argentine Malbec!

Would you like me to suggest a bottle size or provide more information about this wine?


In [46]:
# LLM interaction with ollama lib
from ollama import Client
llm = Client(host="http://localhost:11434/v1")
response = llm.chat(model="llama3.1", messages=[
    {"role": "system", "content": "You are Alfred, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests. You are an expert about wine knowledge and especially good at picking best wines for your users."},
    {"role": "user", "content": "I would like a wine that goes well with steak."},
])
print(response.message.content)

ResponseError: 404 page not found (status code: 404)